# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 1 2023 12:54PM,256396,19,9076481,Eywa Pharma Inc.,Released
1,Feb 1 2023 12:54PM,256396,19,9076495,Eywa Pharma Inc.,Released
2,Feb 1 2023 12:54PM,256396,19,9097772,Eywa Pharma Inc.,Released
3,Feb 1 2023 12:54PM,256396,19,9097773,Eywa Pharma Inc.,Released
4,Feb 1 2023 12:49PM,256401,12,20230102-FL,SugarBear,Released
5,Feb 1 2023 12:42PM,256400,12,HH-39694,Hush Hush,Released
6,Feb 1 2023 12:42PM,256400,12,HH-39695,Hush Hush,Released
7,Feb 1 2023 12:42PM,256400,12,HH-39696,Hush Hush,Released
8,Feb 1 2023 12:42PM,256400,12,HH-39698,Hush Hush,Released
9,Feb 1 2023 12:42PM,256400,12,HH-39699,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,256397,Released,1
22,256398,Released,1
23,256399,Released,22
24,256400,Released,5
25,256401,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
256397,NaN,1.0
256398,NaN,1.0
256399,NaN,22.0
256400,NaN,5.0
256401,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
256340,0.0,1.0
256350,0.0,1.0
256352,17.0,3.0
256353,1.0,0.0
256360,0.0,17.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
256340,0,1
256350,0,1
256352,17,3
256353,1,0
256360,0,17


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,256340,0,1
1,256350,0,1
2,256352,17,3
3,256353,1,0
4,256360,0,17


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,256340,,1
1,256350,,1
2,256352,17,3
3,256353,1,
4,256360,,17


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 1 2023 12:54PM,256396,19,Eywa Pharma Inc.
4,Feb 1 2023 12:49PM,256401,12,SugarBear
5,Feb 1 2023 12:42PM,256400,12,Hush Hush
10,Feb 1 2023 12:31PM,256399,10,ISDIN Corporation
32,Feb 1 2023 12:21PM,256398,10,Emerginnova
33,Feb 1 2023 12:17PM,256397,19,"Granules Pharmaceuticals, Inc."
34,Feb 1 2023 12:15PM,256395,10,ISDIN Corporation
39,Feb 1 2023 11:26AM,256393,10,Eywa Pharma Inc.
52,Feb 1 2023 11:15AM,256392,21,"NBTY Global, Inc."
53,Feb 1 2023 11:12AM,256391,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 1 2023 12:54PM,256396,19,Eywa Pharma Inc.,,4
1,Feb 1 2023 12:49PM,256401,12,SugarBear,,1
2,Feb 1 2023 12:42PM,256400,12,Hush Hush,,5
3,Feb 1 2023 12:31PM,256399,10,ISDIN Corporation,,22
4,Feb 1 2023 12:21PM,256398,10,Emerginnova,,1
5,Feb 1 2023 12:17PM,256397,19,"Granules Pharmaceuticals, Inc.",,1
6,Feb 1 2023 12:15PM,256395,10,ISDIN Corporation,,5
7,Feb 1 2023 11:26AM,256393,10,Eywa Pharma Inc.,11,2
8,Feb 1 2023 11:15AM,256392,21,"NBTY Global, Inc.",,1
9,Feb 1 2023 11:12AM,256391,21,"NBTY Global, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 1 2023 12:54PM,256396,19,Eywa Pharma Inc.,4,
1,Feb 1 2023 12:49PM,256401,12,SugarBear,1,
2,Feb 1 2023 12:42PM,256400,12,Hush Hush,5,
3,Feb 1 2023 12:31PM,256399,10,ISDIN Corporation,22,
4,Feb 1 2023 12:21PM,256398,10,Emerginnova,1,
5,Feb 1 2023 12:17PM,256397,19,"Granules Pharmaceuticals, Inc.",1,
6,Feb 1 2023 12:15PM,256395,10,ISDIN Corporation,5,
7,Feb 1 2023 11:26AM,256393,10,Eywa Pharma Inc.,2,11
8,Feb 1 2023 11:15AM,256392,21,"NBTY Global, Inc.",1,
9,Feb 1 2023 11:12AM,256391,21,"NBTY Global, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 1 2023 12:54PM,256396,19,Eywa Pharma Inc.,4,
1,Feb 1 2023 12:49PM,256401,12,SugarBear,1,
2,Feb 1 2023 12:42PM,256400,12,Hush Hush,5,
3,Feb 1 2023 12:31PM,256399,10,ISDIN Corporation,22,
4,Feb 1 2023 12:21PM,256398,10,Emerginnova,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 1 2023 12:54PM,256396,19,Eywa Pharma Inc.,4.0,NaN
1,Feb 1 2023 12:49PM,256401,12,SugarBear,1.0,NaN
2,Feb 1 2023 12:42PM,256400,12,Hush Hush,5.0,NaN
3,Feb 1 2023 12:31PM,256399,10,ISDIN Corporation,22.0,NaN
4,Feb 1 2023 12:21PM,256398,10,Emerginnova,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 1 2023 12:54PM,256396,19,Eywa Pharma Inc.,4.0,0.0
1,Feb 1 2023 12:49PM,256401,12,SugarBear,1.0,0.0
2,Feb 1 2023 12:42PM,256400,12,Hush Hush,5.0,0.0
3,Feb 1 2023 12:31PM,256399,10,ISDIN Corporation,22.0,0.0
4,Feb 1 2023 12:21PM,256398,10,Emerginnova,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1794700,71.0,25.0
12,512801,6.0,0.0
15,512723,24.0,8.0
16,256340,1.0,0.0
19,1538226,10.0,18.0
21,769173,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1794700,71.0,25.0
1,12,512801,6.0,0.0
2,15,512723,24.0,8.0
3,16,256340,1.0,0.0
4,19,1538226,10.0,18.0
5,21,769173,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,71.0,25.0
1,12,6.0,0.0
2,15,24.0,8.0
3,16,1.0,0.0
4,19,10.0,18.0
5,21,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,71.0
1,12,Released,6.0
2,15,Released,24.0
3,16,Released,1.0
4,19,Released,10.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21
Status,,,,,,
Executing,25.0,0.0,8.0,0.0,18.0,0.0
Released,71.0,6.0,24.0,1.0,10.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21
0,Executing,25.0,0.0,8.0,0.0,18.0,0.0
1,Released,71.0,6.0,24.0,1.0,10.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21
0,Executing,25.0,0.0,8.0,0.0,18.0,0.0
1,Released,71.0,6.0,24.0,1.0,10.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()